In [5]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.models import load_model
import tensorflow as tf
import csv
import pandas as pd
import os 
import json 
import random 

arr_dir = 'model_collocated_10d_filled/'
model = load_model('model-3.keras')
batch_size = 2000
batch_size_val = 500
n_depths = 46
n_depth = 46
num_vars_out = 2
n_var_out = 2
n_var_in = 10
n_samples = 100  # Example sample size

def load_data_from_feather(daily_ds):
    if isinstance(daily_ds, bytes):
        daily_ds = daily_ds.decode('utf-8')    
    df = pd.read_feather('model_collocated_10d_filled/'+daily_ds)
    asal_height_cols = [col for col in df.columns if col.startswith('ASAL_height')]
    ctemp_height_cols = [col for col in df.columns if col.startswith('CTEMP_height')]
    
    other_vars = ['LATITUDE', 'LONGITUDE', 'SSS', 'SST', 'SSH', 'MLD', 'UO', 'VO']
    
    new_df = df[other_vars]
    new_df.dropna(inplace=True)
    new_df = new_df[new_df['LATITUDE'] > -60]
    new_df = new_df[new_df['LATITUDE'] < 60]
    
    height_dfs = []
    for asal_col, ctemp_col in zip(asal_height_cols, ctemp_height_cols):
        height_df = new_df.copy()
        height_df['HEIGHT'] = asal_col.split('_height')[-1]
        height_df['ASAL'] = df[asal_col]  
        height_df['CTEMP'] = df[ctemp_col]  
        height_dfs.append(height_df)
    
    final_df = pd.concat(height_dfs, ignore_index=True)
    final_df = final_df.dropna()

    pivot_table = final_df.pivot_table(index=['LATITUDE', 'LONGITUDE', 'HEIGHT'], values=['SSS', 'SST', 'SSH', 'MLD', 'UO', 'VO', 'ASAL', 'CTEMP'])

    filename = os.path.basename(daily_ds)
    date_str = filename.split('.')[0]  
    date = pd.to_datetime(date_str, format='%Y%m%d')
    day_of_year = date.dayofyear

    pivot_table['DATE'] = day_of_year
    pivot_table = pivot_table.reset_index()
    pivot_table = pivot_table.astype(float)
    pivot_table = pivot_table[['LATITUDE', 'LONGITUDE', 'HEIGHT', 'SSS', 'SST', 'SSH', 'MLD', 'UO', 'VO', 'DATE', 'ASAL', 'CTEMP']]

    cols = ['LATITUDE', 'LONGITUDE', 'SSS', 'SST', 'SSH', 'MLD', 'UO', 'VO', 'DATE', 'HEIGHT']

    groups = pivot_table.groupby(['LATITUDE', 'LONGITUDE'])
    
    arrays = []
    
    n_row = 46
    
    with open('min_max_values.json', 'r') as f:
        min_max_values = json.load(f)
        
    for _, group in groups:
        if len(group) == n_row:
            group_sorted = group.sort_values(by='HEIGHT')
            for column in min_max_values['min_values'].keys():
                min_value = min_max_values['min_values'][column]
                max_value = min_max_values['max_values'][column]
                group_sorted[column] = (group_sorted[column] - min_value) / (max_value - min_value)

            array = group_sorted.values
            if (np.any(array)):
                arrays.append(array)
    
    all_arr = np.stack(arrays)
    X = all_arr[:,:,0:10]
    y = all_arr[:,:,10:12]
    return X, y


def data_generator(feather_files, batch_size):
    for file in feather_files:
        X, y = load_data_from_feather(file)
        num_samples = X.shape[0]
        num_batches = num_samples // batch_size
        for i in range(num_batches):
            start_idx = i * batch_size
            end_idx = (i + 1) * batch_size
            batch_X = X[start_idx:end_idx]
            batch_y = y[start_idx:end_idx]            
            yield batch_X, batch_y

import os 

feather_files = [f for f in os.listdir(arr_dir) if f.endswith('.feather') and os.path.isfile(os.path.join(arr_dir, f))]

start_date = '20100101'
end_date = '20221231'

# To reproduce the code
random.seed(56)
random.shuffle(feather_files)

# Set 0.8 training and 0.2 testing
split_point = int(len(feather_files) * 0.8)  

# Split train/test
training_dates = feather_files[:split_point]
testing_dates = feather_files[split_point:]

split_point_val = int(len(training_dates)*0.9)
validation_dates = training_dates[split_point_val:]
training_dates = training_dates[:split_point_val]

dataset_training = tf.data.Dataset.from_generator(
    data_generator,
    args=[training_dates, batch_size],  # Pass batch_size as an argument
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, n_depth, n_var_in], [None, n_depth, n_var_out])
)

dataset_training = dataset_training.repeat()

dataset_validation = tf.data.Dataset.from_generator(
    data_generator,
    args=[validation_dates, batch_size_val],  # Pass batch_size as an argument
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, n_depth, n_var_in], [None, n_depth, n_var_out])
)

dataset_validation = dataset_validation.repeat()


for batch_X, batch_y in dataset_training.take(1):
    print("Input shape:", batch_X.shape)
    print("Output shape:", batch_y.shape)
    
for batch_X, batch_y in dataset_validation.take(1):
    print("Input shape:", batch_X.shape)
    print("Output shape:", batch_y.shape)

dataset_test = tf.data.Dataset.from_generator(
    data_generator,
    args=[testing_dates, batch_size],  # Pass batch_size as an argument
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, n_depth, n_var_in], [None, n_depth, n_var_out])
)

asal_true_list = []
asal_pred_list = []

ctemp_true_list = []
ctemp_pred_list = []

with open('min_max_values.json', 'r') as f:
        min_max_values = json.load(f)
    
# Iterate over the dataset to collect true and predicted values
for X_batch, y_true_batch in dataset_test:
    y_pred_batch = model.predict(X_batch)

    asal = y_pred_batch[:,:,0] * (min_max_values['max_values']['ASAL'] - min_max_values['min_values']['ASAL']) + min_max_values['min_values']['ASAL']
    ctemp = y_pred_batch[:,:,1] * (min_max_values['max_values']['CTEMP'] - min_max_values['min_values']['CTEMP']) + min_max_values['min_values']['CTEMP']
    
    asal_pred_list.append(asal)
    ctemp_pred_list.append(ctemp)

    asal = y_true_batch[:,:,0] * (min_max_values['max_values']['ASAL'] - min_max_values['min_values']['ASAL']) + min_max_values['min_values']['ASAL']
    ctemp = y_true_batch[:,:,1] * (min_max_values['max_values']['CTEMP'] - min_max_values['min_values']['CTEMP']) + min_max_values['min_values']['CTEMP']

    asal_true_list.append(asal)
    ctemp_true_list.append(ctemp)


asal_true = np.concatenate(asal_true_list, axis=0)
asal_pred = np.concatenate(asal_pred_list, axis=0)

ctemp_true = np.concatenate(ctemp_true_list, axis=0)
ctemp_pred = np.concatenate(ctemp_pred_list, axis=0)



data = []
for i in range(n_depths):
    # Calculate metrics for asal
    asal_mse = mean_squared_error(asal_true[:, i], asal_pred[:, i])
    asal_mae = mean_absolute_error(asal_true[:, i], asal_pred[:, i])
    asal_r2 = r2_score(asal_true[:, i], asal_pred[:, i])
    asal_variance = np.var(asal_pred[:, i])

    # Calculate metrics for ctemp
    ctemp_mse = mean_squared_error(ctemp_true[:, i], ctemp_pred[:, i])
    ctemp_mae = mean_absolute_error(ctemp_true[:, i], ctemp_pred[:, i])
    ctemp_r2 = r2_score(ctemp_true[:, i], ctemp_pred[:, i])
    ctemp_variance = np.var(ctemp_pred[:, i])

    # Append the metrics to the data list
    data.append([asal_mse, asal_mae, asal_r2, asal_variance, ctemp_mse, ctemp_mae, ctemp_r2, ctemp_variance])

# Write the data to a CSV file
with open('metrics_model-3.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ASAL MSE', 'ASAL MAE', 'ASAL R2', 'ASAL Variance', 'CTEMP MSE', 'CTEMP MAE', 'CTEMP R2', 'CTEMP Variance'])
    writer.writerows(data)
    
import numpy as np
print(mean_squared_error(asal_true[:, :], asal_pred[:, :]))
print(mean_absolute_error(asal_true[:, :], asal_pred[:, :]))
print(r2_score(asal_true[:, :], asal_pred[:, :]))

print(mean_squared_error(ctemp_true.ravel(), ctemp_pred.ravel()))
print(mean_absolute_error(ctemp_true.ravel(), ctemp_pred.ravel()))
print(r2_score(ctemp_true.ravel(), ctemp_pred.ravel()))

/tmp/ipykernel_1166516/3616892311.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.dropna(inplace=True)


Input shape: (2000, 46, 10)
Output shape: (2000, 46, 2)


2024-05-30 06:25:29.244950: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/tmp/ipykernel_1166516/3616892311.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.dropna(inplace=True)


Input shape: (500, 46, 10)
Output shape: (500, 46, 2)


2024-05-30 06:25:37.677084: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/tmp/ipykernel_1166516/3616892311.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.dropna(inplace=True)
/tmp/ipykernel_1166516/3616892311.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.dropna(inplace=True)


...

63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step
 2/63 ━━━━━━━━━━━━━━━━━━━━ 5s 83ms/step

/tmp/ipykernel_1166516/3616892311.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.dropna(inplace=True)


63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step


2024-05-30 10:08:44.182061: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


0.09700818
0.24037895
0.9137001823272974
0.6669631
4.511024
1.1685253
0.7844636066039741
68.18278


In [18]:
print(mean_squared_error(ctemp_true, ctemp_pred))
print(mean_absolute_error(ctemp_true, ctemp_pred))
print(r2_score(ctemp_true, ctemp_pred))
print(np.var(ctemp_pred))

4.511024
1.1685253
0.7844636066039741
68.18278


In [17]:
print(mean_squared_error(asal_true, asal_pred))
print(mean_absolute_error(asal_true, asal_pred))
print(r2_score(asal_true, asal_pred))
print(np.var(ctemp_pred))

0.09700818
0.24037895
0.9137001823272974
68.18278


In [7]:
ctemp_pred.ravel()

array([4.2599945, 3.9734104, 4.027628 , ..., 2.7501867, 2.5710843,
       2.4000294], dtype=float32)

In [9]:
data = []
# Loop through each depth
for i in range(n_depths):
    asal_mse = mean_squared_error(asal_true[:, i].ravel(), asal_pred[:, i].ravel())
    asal_mae = mean_absolute_error(asal_true[:, i].ravel(), asal_pred[:, i].ravel())
    asal_r2 = r2_score(asal_true[:, i].ravel(), asal_pred[:, i].ravel())
    asal_variance = np.var(asal_pred[:, i].ravel())

    ctemp_mse = mean_squared_error(ctemp_true[:, i].ravel(), ctemp_pred[:, i].ravel())
    ctemp_mae = mean_absolute_error(ctemp_true[:, i].ravel(), ctemp_pred[:, i].ravel())
    ctemp_r2 = r2_score(ctemp_true[:, i].ravel(), ctemp_pred[:, i].ravel())
    ctemp_variance = np.var(ctemp_pred[:, i].ravel())

    data.append([asal_mse, asal_mae, asal_r2, asal_variance, ctemp_mse, ctemp_mae, ctemp_r2, ctemp_variance])

with open('metrics_model-0-v2.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ASAL MSE', 'ASAL MAE', 'ASAL R2', 'ASAL Variance', 'CTEMP MSE', 'CTEMP MAE', 'CTEMP R2', 'CTEMP Variance'])
    writer.writerows(data)